In [1]:
import datetime
import inspect
import numpy as np
import pandas as pd

from pynwb import NWBHDF5IO, NWBFile
from pynwb.core import DynamicTableRegion, DynamicTable
from pynwb.device import Device

#from ndx_stimulation import StimulusFunctionTable
import os

from pynwb import register_class, load_namespaces
from hdmf.utils import docval, call_docval_func, getargs, get_docval


In [2]:
# Set path of the namespace.yaml file to the expected install location                                                  
#ndx_stimulation_specpath = os.path.join(
#    os.path.dirname(__file__),
#    'spec',
#    'ndx-stimulation.namespace.yaml'
#)
ndx_stimulation_specpath = '/Users/pam.baker/Documents/ndx_stim/ndx-stimulation/spec/ndx-stimulation.namespace.yaml'

In [3]:
# Load the namespace                                                                                                 
load_namespaces(ndx_stimulation_specpath)

{'ndx-stimulation': {'core': ('Container', 'DynamicTable'),
  'hdmf-common': ('VectorIndex', 'VectorData')}}

In [4]:
@register_class('StimulusFunctionTable', 'ndx-stimulation')
class StimulusFunctionTable(DynamicTable):   
    
    __columns__ = ( 
                    {'name':'function_name', 
                     'description': 'The names of the 1D stimulus waveforms.',
                     'required': True,
                     'index': False},
                    {'name':'float_parameters', 
                     'description': 'The names of the float parameters for the 1D stimulus waveforms.',
                     'required': True,
                     'index': True},
                    {'name':'function_parameters',
                     'description': 'The function parameters for the 1D stimulus waveforms.',
                     'required': True,
                     'index': True}
    )
    
    
    @docval(*get_docval(DynamicTable.__init__, 'id', 'columns', 'colnames'))
    def __init__(self, **kwargs):
        kwargs['name'] = ('Stimulus Function Table')
        kwargs['description'] = ('Table for storing ontologized 1D stimulus waveform metadata')
        call_docval_func(super().__init__, kwargs)
        

In [5]:
sft = StimulusFunctionTable()

In [6]:
float_params1 = [('amplitude', -0.110, 'V'), 
                 ('duration', 0.500, 's'),
                 ('start_time', 0.1, 's')
                ]

float_params2 = [('amplitude', -0.090, 'V'), 
                 ('duration', 0.500, 's'),
                 ('start_time', 0.1, 's')
                ]

wave1 = {'function_name':'sq', 'float_parameters': float_params1, 'function_parameters':[]}
wave2 = {'function_name':'sq', 'float_parameters': float_params2, 'function_parameters':[]}

sft.add_row(data = wave1)
sft.add_row(data = wave2)

In [11]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [14]:
pd.set_option("display.max_colwidth", 100)

display(sft.to_dataframe())

,function_name,float_parameters,function_parameters
id,,,
0,sq,"[(amplitude, -0.11, V), (duration, 0.5, s), (start_time, 0.1, s)]",[]
1,sq,"[(amplitude, -0.09, V), (duration, 0.5, s), (start_time, 0.1, s)]",[]


In [9]:
print(sft.colnames)

('function_name', 'float_parameters', 'function_parameters')


In [15]:
sft['float_parameters']

In [33]:
sft.get('float_parameters')

In [16]:
from datetime import datetime
from dateutil.tz import tzlocal

start_time = datetime(2017, 4, 3, 11, tzinfo=tzlocal())
create_date = datetime(2017, 4, 15, 12, tzinfo=tzlocal())


nwbfile = NWBFile('demonstrate caching', 'NWB456', start_time,
                  file_create_date=create_date)

nwbfile.add_acquisition(sft)


In [17]:
# Write the SFT out to file

from pynwb import NWBHDF5IO

io = NWBHDF5IO('sft_cache_spec_example.nwb', mode='w')
io.write(nwbfile)
io.close()

In [39]:
# Reading in the file I just wrote

io = NWBHDF5IO('sft_cache_spec_example.nwb', mode='r', load_namespaces=True)
sft_nwbfile = io.read()


/opt/anaconda3/lib/python3.7/site-packages/hdmf/spec/namespace.py:470: UserWarning: ignoring namespace 'hdmf-common' because it already exists
  warn("ignoring namespace '%s' because it already exists" % ns['name'])
/opt/anaconda3/lib/python3.7/site-packages/hdmf/spec/namespace.py:470: UserWarning: ignoring namespace 'core' because it already exists
  warn("ignoring namespace '%s' because it already exists" % ns['name'])
/opt/anaconda3/lib/python3.7/site-packages/hdmf/spec/namespace.py:470: UserWarning: ignoring namespace 'ndx-stimulation' because it already exists
  warn("ignoring namespace '%s' because it already exists" % ns['name'])


In [64]:
# print(sft_nwbfile.acquisition)

sft_in = sft_nwbfile.get_acquisition()

In [42]:
sft_in.to_dataframe()

,function_name,float_parameters,function_parameters
id,,,
0,sq,"[[amplitude, 0.1, V], [duration, 0.5, s]]",[]


In [98]:
sft_in['float_parameters'].data[:][0]

2